# Atividade 3

## Redes Neurais Recorrentes

Discutimos como uma rede neural recorrente, através de suas células de memória, permitem processamento de dados sequencial. Também falamos sobre embedding e fizemos um exercício para entender o objetivo e função de um Word2Vec.

A biblioteca Keras traz acesso alto nível a camadas que permitem a fácil implementação deste tipo de layer.

Este tipo de técnica é boa para generalizar informação esparsa condensando em uma camada densa:

![word embeddings vs. one hot encoding](https://s3.amazonaws.com/book.keras.io/img/ch6/word_embeddings.png)

Vamos ver exemplos de implementação destas camadas:

In [ ]:
import tensorflow

from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, CuDNNLSTM
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

#### Stacks de RNNs

Em alguns casos, é interessante criar uma sequência de camadas recorrentes para processamento da informação. Nestes casos é necessário retornar as sequências para compartilhar com suas camadas vizinhas, com exceção da camada final. Este processo é feito passando o valor booleano `True` no o parâmetro `return_sequences` destas camadas, veja exemplo:

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))  # Este layer final apenas retorna os últimos outputs.
model.summary()

## Análise e classificação de texto

Nesta atividade utilizaremos um _dataset_ chamado __IMDB__ composto de opiniões sobre filmes em formato textual. O objetivo é classificar estes _inputs_ de forma binária (0 e 1) onde identificaremos se a opinião é _**positiva**_ ou _**negativa**_.

__IMDB__ (Internet Movie DataBase) conta com um total de 50.000 opiniões sobre filmes onde separamos 25.000 delas para treino e os 50% restantes para teste. As frações são balanceadas, ou seja, contem um número igual de opiniões positivas e negativas.

In [ ]:
from tensorflow.keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000 # Número máximo de palavras consideradas como features
max_len      = 500 # Tamanho máximo do texto utilizado como input
batch_size   = 32 # Tamanho do batch de processamento

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'sequências de treino')
print(len(x_test), 'sequências de teste')

print('PadSequences (amostras x tamanho)')
input_train = sequence.pad_sequences(x_train, maxlen=max_len)
input_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

#### PadSequence

Precisamos formatar um padrão de input para nossa rede. Definimos que nosso input máximo são 500 palavras. O que este processo faz é cortar textos maiores e preencher com 0 o que falta para completar 500 posições em textos menores que 500 palavras em seu conteúdo.

Veja exemplo:

In [ ]:
print(len(x_train[0]))
x_train[0][:10]

In [ ]:
input_train[0]

Para ter ideia do conteúdo deste dado, é possível baixar o dicionário representativo da informação. Da mesma maneira que fizemos na aula passada, podemos consultar o valor de cada número neste dicionário.

Veja o exemplo:

In [ ]:
dic_palavras = imdb.get_word_index()

In [ ]:
dic_palavras['woody']

Este dicionário possui o valor da palavra como chave, podemos inverter seu conteúdo de `palavra : número` para `número : palavra` para facilitar nosso processamento:

In [ ]:
dic_palavras_formatado = dict([(valor, chave) for (chave, valor) in dic_palavras.items()])

In [ ]:
dic_palavras_formatado[2289]

Agora podemos iterar sobre o vetor de _input_ traduzindo a informação, vejamos o exemplo na posição `0`:

In [ ]:
texto = []

for item in x_train[0]:
    texto.append(dic_palavras_formatado.get(item, '?'))
    
print(texto)
print('------------------*------------------')
print(' '.join(texto))

### Treinando a rede

Podemos treinar uma rede de arquitetura simples para resolver o problema acima utilizando as camadas de `Embedding` e `SimpleRNN`:

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

#### Resultados:

In [ ]:
[loss, acc] = model.evaluate(input_test, y_test)
print("Acc: {:.4f}".format(acc))

acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Acc no treino')
plt.plot(epochs, val_acc, label='Acc da validação')
plt.title('Acc no treino e validação')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label='Loss no treino')
plt.plot(epochs, val_loss, label='Loss na validação')
plt.title('Loss no treino e validação')
plt.legend()

plt.show()

In [ ]:
test_preds = model.predict(input_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

plt.figure(figsize = (15, 10))
plt.plot(fpr, tpr, label='Modelo (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Curva ROC')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--')

plt.show()

### LSTM

Vamos para um exemplo mais objetivo, vamos implementar a mesma arquitetura utilizando uma camada de **LSTM**.

Keras traz uma implementação para processamento em GPUs com arquitetura CUDA. Se em seu ambiente, você possui uma GPU com este tipo de arquitetura, pode utilizar a classe `CuDNNLSTM`.

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

#### Resultados:

In [ ]:
[loss, acc] = model.evaluate(input_test, y_test)
print("Acc: {:.4f}".format(acc))

acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Acc no treino')
plt.plot(epochs, val_acc, label='Acc da validação')
plt.title('Acc no treino e validação')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label='Loss no treino')
plt.plot(epochs, val_loss, label='Loss na validação')
plt.title('Loss no treino e validação')
plt.legend()

plt.show()

In [ ]:
test_preds = model.predict(input_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

plt.figure(figsize = (15, 10))
plt.plot(fpr, tpr, label='Modelo (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Curva ROC')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--')

plt.show()

## Exercício Proposto

Explore outras arquiteturas de rede e veja se você consegue obter melhores resultados com o exemplo visto acima.

In [ ]:
model2 = Sequential()
model2.add(Embedding(max_features, 32))
model2.add(LSTM(32, return_sequences=True))
model2.add(LSTM(32, return_sequences=True))
model2.add(LSTM(32))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(input_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

In [ ]:
[loss, acc] = model.evaluate(input_test, y_test)
print("Acc: {:.4f}".format(acc))

acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Acc no treino')
plt.plot(epochs, val_acc, label='Acc da validação')
plt.title('Acc no treino e validação')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label='Loss no treino')
plt.plot(epochs, val_loss, label='Loss na validação')
plt.title('Loss no treino e validação')
plt.legend()

plt.show()

In [ ]:
test_preds = model.predict(input_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

plt.figure(figsize = (15, 10))
plt.plot(fpr, tpr, label='Modelo (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Curva ROC')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--')

plt.show()

## Leitura

### Um problema de previsão

Problemas envolvendo sequências podem ser encontrados em outros formatos. No meio médico; Sequências de imagens em um exame de tomografia ou valores de oxigenação de um paciente monitorado, são exemplos.

Pensando em negócio, um modelo capaz de prever temperatura em um certo período pode ser utilizado como feature em um motor de tomada de decisão. Dentro do meio hospitalar, pode ser utilizado como feature para prever a demanda diária de uma unidade de pronto atendimento.

O artigo em anexo abaixo faz parte do livro [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff) e traz um exemplo de previsão utilizando um conjunto de informações coletadas na estação meteorológica do [Instituto Max-Plank para biogeoquímica em Jena na Alemanha](http://www.bgc-jena.mpg.de/wetter/).

Além de trazer o uso de LSTMs bidirecionais, o que veremos na próxima aula deste curso, mostra um exemplo de consumo sequencial de informação pela rede.

[Link para download da base](https://www.kaggle.com/stytch16/jena-climate-2009-2016)
[Link para o artigo](https://github.com/pgiaeinstein/nlp/blob/master/fcholletRNNS.ipynb)

In [ ]:
import pandas as pd

data = pd.read_excel('https://raw.githubusercontent.com/pgiaeinstein/nlp/master/exemplo_prox_aula.xlsx')

## Problemas futuros

Em nossa próxima aula, estudaremos a utilização das ConvNets nas camadas de uma rede neural. Trabalharemos com imagens e novamente com texto. O objetivo é modelar uma rede capaz de extrair entidades dos textos que processamos nas atividades anteriores desta aula.

Para exemplificar, veja a tabela abaixo. Vimos uma forma de encontrar padrões textuais utilizando expressões regulares, o que nos possibilita segmentar o texto classificando cada palavra.

Vamos ver uma abordagem de resolver o problema através de uma rede neural.

In [ ]:
data


# Lab 01

## Introdução

O objetivo deste laboratório é explorar diferentes técnicas utilizadas para classificação em aprendizado de máquina, utilizaremos modelos clássicos de classificação e introduziremos um modelo simples de rede neural para resolver o mesmo problema.

O intuito desta atividade é a familiarização das bibliotecas em Python utilizadas por padrão na análise de dados assim como mostrar que o aprendizado de máquina é algo simples e acessível a qualquer um.

## Sobre o DataSet

Para este laboratório, vamos brincar com o dataset [Breast Cancer Wisconsin](http://mlr.cs.umass.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29).

Este dataset é público e foi disponibilizado em novembro de 1995, o objetivo é classificar tumores como benignos ou malignos considerando valores obtidos por análise de imagem.

Mais informações sobre este dataset e seus valores podem ser consultadas [neste link](http://mlr.cs.umass.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names).

## Bibliotecas utilizadas

### NumPy

[NumPy](http://www.numpy.org/) é uma famosa biblioteca utilizada para fins científicos, facilita a criação, manipulação e cálculos envolvendo vetores e matrizes.

### Pandas

[Pandas](https://pandas.pydata.org/) provê uma interface que nos permite manipular dados de forma similar ao que faríamos utilizando uma tabela de Excel. Nos devolve uma estrutura de dados chamada [DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html), onde organizamos os dados em linhas e colunas.

### MatPlotLib

[MatPlotLib](https://matplotlib.org/) é uma biblioteca para plotagem de gráficos. Suas ferramentas permitem customização completa dos gráficos gerados.

### Seaborn

[Seaborn](https://seaborn.pydata.org/) provê uma camada _high-level_ de abstração para a utilização da biblioteca Matplotlib, ou seja, é um facilitador.

### Scikit-Learn

[Scikit-Learn](http://scikit-learn.org/stable/) é uma das principais bibliotecas utilizadas para machine learning em Python, é open source e mantida por diversas instituições de ensino.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

### Importando os dados para uma estrutura de DataFrame

A biblioteca Pandas possui métodos facilitadores para a importação de vários tipos de fontes de dados em um DataFrame. Neste laboratório vamos utilizar um arquivo no formato csv ([**C**omma-**S**eparated **V**alues](https://pt.wikipedia.org/wiki/Comma-separated_values)).

Para ter uma visão de todos os facilitadores de importação como quais parâmetros de formatação podemos utilizar para importar estes dados, veja a [documentação da biblioteca](https://pandas.pydata.org/pandas-docs/stable/io.html).

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pgiaeinstein/otmz-mlp/master/bcw.data.csv', sep=',')
df

Veja que quando imprimo o dataframe completo ele me mostra uma quantidade de 60 itens sendo os 30 primeiros e os 30 últimos da coleção.

Essa forma de visualizar nem sempre é necessária e pode poluir nossa documentação. Para uma visão mais controlada do Dataframe, podemos utilizar o método [`head()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) que imprime, por padrão, as primeiras 5 linhas de informação.

In [ ]:
df.head()

O método aceita como argumento principal o número de linhas que desejamos imprimir, veja no exemplo abaixo quando solicitamos que as 32 linhas iniciais sejam impressas em nosso documento.

In [ ]:
df.head(32)

### Modificando o dataframe

Para uma análise inicial dos dados, outro método interessante é o [`info()`](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.info.html), este método imprime um resumo quantitativo e qualitativo além da estrutura completa de nosso Dataframe.

In [ ]:
df.info()

Vamos entender o que é impresso acima:

Temos 569 linhas de informação neste dataframe, iniciando no índice 0 até o índice 568. Nestas 569 entradas temos 33 atributos por linha, ou seja, temos 33 colunas.

Verifique que o método sumariza para cada linha, o nome de sua coluna, o total de valores não nulos nesta coluna assim como também o tipo de dado que a coluna guarda.

Repare na coluna `Unnamed: 32`; Está coluna possui 0 valores não nulos, ou seja, todos os valores desta coluna são nulos e devem ser desconsiderados pois não nos ajuda em nada neste laboratório.

Além da coluna `Unnamed: 32`, este dataframe possui outra coluna `id` que não faz sentido para este laboratório.

Por último, temos um resumo dos tipos de dados presentes no dataframe e também qual o tamanho em memória ocupado por este dataframe.

O método [`drop()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html) permite eliminar colunas ou linhas do nosso dataframe, no caso, vamos remover as colunas `Unnamed: 32` e `id` como dito acima e utilizaremos o argumento `inplace = True` para modificar o objeto em que estamos utilizando o método.

Por segurança, vamos criar uma cópia do objeto original e salvaremos na variável `df_inicial`.

In [ ]:
# vou criar uma cópia do dataframe inicial por segurança
df_inicial = df

# vamos remover as colunas
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)
df.head()

Reparem na coluna `diagnosis`, esta coluna é chamada de *TARGET*, ou seja, é a nossa coluna de saída para nosso modelo de classificação.

Ela possui dois valores em formato `char` (**M** e **B**), para facilitar nossa vida, vamos modificar essa variável para um valor numérico.

Criaremos um dicionário auxiliar para o método [`map()`](http://book.pythontips.com/en/latest/map_filter.html) onde vamos classificar a letra **B** (Benigno) como **0** e a letra **M** (Maligno) como **1**.

In [ ]:
label = {
    'B' : 0,
    'M' : 1
}

df['diagnosis'] = df['diagnosis'].map(label)
df

Depois desta transformação, agora nossa coluna de saída possui valores numéricos distintos (0 e 1).

Vamos analisar de forma mais estatística nosso dataframe agora, para isso, utilizamos o método [`describe()`](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.describe.html).

Este método retorna um resumo estatístico de nosso dataframe, por coluna, desconsiderando valores nulos.

In [ ]:
df.describe()

### Escolhendo nossas Features

Temos 3 tipos de medidas neste dataset: Mean, SE e Worst. É importante entender como estes valores se comunicam.

Para alguns modelos clássicos utilizados em aprendizado de máquina, temos problemas quanto maior for o número de entradas e de colunas, característica esta que é o inverso quando comparada a um modelo de rede neural, por exemplo.

Entendendo isso vamos escolher variáveis de *input* que beneficiem nossa tarefá de classificação, sem utilizar nenhum modelo auxiliar para selecionar estas variáveis.

Separamos então nossas colunas em 3 tipos distintos: Mean, SE e Worst. Após essa separação, vamos analisar cada grupo buscando possíveis correlações entre estas variáveis.

In [ ]:
# vamos criar um dataframe excluindo a coluna 'diagnosis':
df_parcial = df.iloc[:, 1:]
df_parcial.head()

In [ ]:
# Neste novo dataframe, vamos listar as colunas que restaram:
df_parcial.columns

Verifiquem que os 3 grupos de medidas estão em ordem, temos então levando como base o array acima:

#### Variáveis do tipo MEAN:
> Posição 0 até posição 9 do array.

#### Variáveis do tipo SE:
> Posição 10 até posição 19 do array.

#### Variáveis do tipo WORST:
> Posição 20 até posição 29 do array.

Vamos separar nossas features por tipo, criando listas com as colunas de interesse para cada tipo de variável.

In [ ]:
f_mean  = list(df_parcial.columns[:10])
f_se    = list(df_parcial.columns[10:20])
f_worst = list(df_parcial.columns[20:])

In [ ]:
f_mean

In [ ]:
f_se

In [ ]:
f_worst

Outro aspecto importante que devemos sempre levar em consideração é como as colunas se correlacionam, uma correlação forte entre variáveis tende a divergir o resultado do modelo em alguns casos.

Agora vamos obter a matriz de correlação destas variáveis.

In [ ]:
correlacao = df_parcial[f_mean].corr()
correlacao

Lembrando que quanto maior a proximidade do valor entre 1 e -1, maior é a correlação entre as duas colunas para facilitar a visualização desta matriz, algo que é muito utilizado é um gráfico do tipo HeatMap.

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(correlacao, xticklabels = f_mean, yticklabels= f_mean, cbar = True, square = True, annot = True, fmt = '.2f', annot_kws={ 'size' : 15}, cmap = 'winter', ax = ax)

#### Interpretando o gráfico

As observações aqui são simples, vamos remover de nossa base, todos os campos que tem forte correlação!

Verifique que existem 2 "quadrados" onde é possível verificar forte correlação entre os campos: `radius_mean`, `perimeter_mean` e `area_mean` formam o primeiro "quadrado" e os campos `compactness_mean`, `concavity_mean` e `concavepoint_mean` formam o segundo.

Destes dois conjuntos, escolhemos um de cada e seguimos com nossa analise.

Do primeiro grupo, vemos que `area_mean` tem os menores valores de correlação com as demais colunas, vamos escolher esta *feature* neste conjunto.

Do segundo conjunto, vemos que `compactness_mean` tem os menores valores, então seguiremos com ele.

Nossa lista de colunas final será:

In [ ]:
features_mean = ['texture_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'symmetry_mean', 'fractal_dimension_mean']

#### Criando dataframes separados entre features e meta

Como já temos nossas features iniciais, podemos agora criar dois vetores, um com nossas *features* escolhidas e outro chamado de *target*, ou seja, com a classificação para cada linha de nosso dataset.

In [ ]:
features = df[features_mean]
features.head()

In [ ]:
target = df['diagnosis']
target.head()

#### Separando nossa amostra em treino e teste

Vamos separar agora a nossa base entre uma base de treino e uma base de teste.

In [ ]:
seed = 4

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size = 0.2, random_state = seed)

Vamos criar duas funções que irão nos auxiliar com o treino, a predição e a exibição dos resultados de nossas predições.

In [ ]:
def calcula_resultados(pred_output, real_output):
    cm  = confusion_matrix(real_output, pred_output)
    acc = accuracy_score(real_output, pred_output)
    f1  = f1_score(real_output, pred_output)
    ps  = precision_score(real_output, pred_output)
    rs  = recall_score(real_output, pred_output)
    
    return {
        'matrix'   : cm,
        'accuracy' : acc,
        'f1'       : f1,
        'ps'       : ps,
        'rs'       : rs
    }

def testa_modelo(modelo, X_train, X_test, Y_train, Y_test):
    modelo.fit(X_train, Y_train)
    pred_output = modelo.predict(X_test)
    response = calcula_resultados(pred_output, Y_test)
    
    print('-----------------------------')
    print('Accuracy : {}'.format(response['accuracy']))
    print('F1 : {}'.format(response['f1']))
    print('Precision : {}'.format(response['ps']))
    print('Recall : {}'.format(response['rs']))
    print('-----------------------------')
    sns.heatmap(response['matrix'], annot = True, cmap = 'winter')

### Modelos

#### SVM (Support Vector Machine)

Uma [**SVM**](https://pt.wikipedia.org/wiki/M%C3%A1quina_de_vetores_de_suporte) é um excelente método para se testar em primeiro lugar quando não se tem nenhum conhecimento prévio sobre um domínio. Três propriedades tornam a SVM atraente:

1. Constroem um **separador de margem máxima**:

![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/svm01.jpg)
![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/svm02.png)

2. Criam uma separação linear em hiperplano, mas tem a capacidade de entender dados em um espaço de dimensão superior, usando o **truque de kernel**.

![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/svm03.png)

3. Uma SVM é **não paramétrica**, ou seja, existe a necessidade em guardar os exemplos de treinamento. Porém, na prática, acabam guardando apenas uma **pequena fração do número de exemplos**.

![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/svm04.png)

In [ ]:
svc_model = SVC()
testa_modelo(svc_model, X_train, X_test, Y_train, Y_test)

#### Padronização dos dados

Já discutimos que alguns algoritmos sofrem com dados em escalas que divergem muito, vamos ver este conceito na prática.

Reparem na distribuição de nossas features atualmente:

In [ ]:
X_train.describe()

A padronização realiza a seguinte operação:
    
$$ 
X_i = \frac{X_i \times \overline{X}}{std_X}
$$

Basicamente o que estamos realizando é ignorar a distribuição original da nossa base. Transformaremos os dados para obter uma média muito próxima de 0 e desvio padrão próximo de 1, sendo assim não teremos valores com grande variância na nossa base.

In [ ]:
scaler = StandardScaler()

X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [ ]:
X_train_scaler_df = pd.DataFrame(X_train_scaler, columns = X_train.columns)
X_train_scaler_df.describe()

In [ ]:
testa_modelo(svc_model, X_train_scaler, X_test_scaler, Y_train, Y_test)

#### Árvore de Decisão

Uma **Árvore de Decisão** representa uma função que recebe em seus parâmetros de entrada um vetor de valores e retorna uma resposta / classificação.

![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/DT01.png)

Uma árvore alcança sua resposta executando uma sequência de testes onde cada nó interno de sua estrutura corresponde a um teste do valor e de um dos atributos de entrada.

In [ ]:
random_state = 0

DT_model = DecisionTreeClassifier(random_state = random_state)
testa_modelo(DT_model, X_train, X_test, Y_train, Y_test)

In [ ]:
testa_modelo(DT_model, X_train_scaler, X_test_scaler, Y_train, Y_test)

#### kNN (K Nearest-Neighbor)

O **kNN** ([k-Vizinhos Mais Próximos](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)) é um algoritmo não linear que literalmente mede a distância de um determinado ponto sem classificação em relação a k pontos conhecidos.

A classificação deste ponto então se dá pelo maior número de similares dentre os k vizinhos mais próximos analisados:

![SVM01](https://github.com/pgiaeinstein/otmz-mlp/raw/master/img/knn01.png)

A distância pode ser calculada de vários modos, o mais comum é utilizar a distância euclidiana, que respeita a seguinte equação:

$$
D_{A,B} = \sqrt{(A_1 - B_2)^2+(A_2 - B_2)^2+\ldots+(B_n - B_n)^2}
$$

In [ ]:
knn_model = KNeighborsClassifier()
testa_modelo(knn_model, X_train, X_test, Y_train, Y_test)

In [ ]:
testa_modelo(knn_model, X_train_scaler, X_test_scaler, Y_train, Y_test)

Novamente vemos um resultado melhor quando utilizamos os dados padronizados.

## Exercício 2

Crie uma rede neural para classificar o problema proposto no **Lab01**, compare os valores obtidos anteriormente com o melhor valor encontrado em sua rede.

In [ ]:
# Resolução





### Exercício 3

In [ ]:
data_ns = pd.read_csv('https://raw.githubusercontent.com/pgiaeinstein/otmz-mlp/master/base_ns.csv', sep=',')

In [ ]:
data_ns.head()